In [48]:
%load_ext autoreload
%autoreload 2

frozenset({'type_1_m5', 'm5', 'type_2_d', 'type_1_w', 'M', 'type_2_m15', 'type_2_M', 'type_1_m15', 'h1', 'm15', 'type_2_h4', 'type_1_h1', 'd', 'm30', 'type_2_w', 'type_2_m5', 'type_1_d', 'h4', 'type_1_M', 'm1', 'type_2_m30', 'w', 'type_2_m1', 'type_1_h4', 'type_2_h1', 'type_1_m30', 'type_1_m1'})
[Trade(uid='1', tags=[Tag(timestamp=None, key='type_1_m1', value=True), Tag(timestamp=None, key='type_3_m15', value=True), Tag(timestamp=None, key='entry_price', value=1.1), Tag(timestamp=None, key='sl_price', value=1.095), Tag(timestamp=None, key='tp_price', value=1.11), Tag(timestamp=None, key='SL_distance', value=0.0050000000000001155), Tag(timestamp=None, key='TP_distance', value=0.010000000000000009), Tag(timestamp=None, key='entry_time', value=datetime.datetime(2024, 12, 28, 17, 4, 22, 660157)), Tag(timestamp=None, key='risk_reward_ratio', value=1.9999999999999556)]), Trade(uid='2', tags=[Tag(timestamp=None, key='type_2_h1', value=True), Tag(timestamp=None, key='entry_price', value=1.2), 

In [58]:
from src.tradecli import TradeJournal
from src.journal import journal

frozenset({'type_1_m5', 'm5', 'type_2_d', 'type_1_w', 'M', 'type_2_m15', 'type_2_M', 'type_1_m15', 'h1', 'm15', 'type_2_h4', 'type_1_h1', 'd', 'm30', 'type_2_w', 'type_2_m5', 'type_1_d', 'h4', 'type_1_M', 'm1', 'type_2_m30', 'w', 'type_2_m1', 'type_1_h4', 'type_2_h1', 'type_1_m30', 'type_1_m1'})
[Trade(uid='1', tags=[Tag(timestamp=None, key='type_1_m1', value=True), Tag(timestamp=None, key='type_3_m15', value=True), Tag(timestamp=None, key='entry_price', value=1.1), Tag(timestamp=None, key='sl_price', value=1.095), Tag(timestamp=None, key='tp_price', value=1.11), Tag(timestamp=None, key='SL_distance', value=0.0050000000000001155), Tag(timestamp=None, key='TP_distance', value=0.010000000000000009), Tag(timestamp=None, key='entry_time', value=datetime.datetime(2024, 12, 28, 18, 3, 13, 47076)), Tag(timestamp=None, key='risk_reward_ratio', value=1.9999999999999556), Tag(timestamp=None, key='session', value='new_york')]), Trade(uid='2', tags=[Tag(timestamp=None, key='type_2_h1', value=True)

In [59]:
# Generate the feature dataframe
df = journal.to_dataframe()
df

,trade_uid,close_price,TP_distance,tp_price,risk_reward_ratio,type_3_m15,session,type_2_h1,entry_price,SL_distance,management_strategy,sl_price,entry_time,type_1_m1
0,1,NaN,0.01,1.11,2.000000,True,new_york,None,1.1,0.005,None,1.095,2024-12-28 18:03:13.047076,True
1,2,NaN,0.01,1.21,0.666667,None,new_york,True,1.2,0.015,None,1.185,2024-12-28 18:03:13.047096,None
2,3,NaN,0.03,1.33,10.000000,None,london,None,1.3,0.500,None,1.297,2024-12-27 10:00:01.000000,None
3,4,1.305,0.03,1.33,10.000000,None,london,None,1.3,0.500,strategy_2,1.297,2024-12-25 10:00:00.000000,None


In [61]:
# Example analysis: Compare trades with different SL distances
sl_trades = df[df['SL_distance'] > 0]
print(sl_trades)

  trade_uid  close_price  TP_distance  tp_price  risk_reward_ratio type_3_m15  \
0         1          NaN         0.01      1.11           2.000000       True   
1         2          NaN         0.01      1.21           0.666667       None   
2         3          NaN         0.03      1.33          10.000000       None   
3         4        1.305         0.03      1.33          10.000000       None   

    session type_2_h1  entry_price  SL_distance management_strategy  sl_price  \
0  new_york      None          1.1        0.005                None     1.095   
1  new_york      True          1.2        0.015                None     1.185   
2    london      None          1.3        0.500                None     1.297   
3    london      None          1.3        0.500          strategy_2     1.297   

                  entry_time type_1_m1  
0 2024-12-28 18:03:13.047076      True  
1 2024-12-28 18:03:13.047096      None  
2 2024-12-27 10:00:01.000000      None  
3 2024-12-25 10:00:00.00